In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,substring
from pyspark.sql import functions as F

In [4]:
spark = SparkSession.builder.appName('Lowest Revenue Generated Restaurants').getOrCreate()

In [5]:
df_input = spark.read.options(header='True', InferSchema='True').csv('doordash_delivery.csv')
#df_input.show(10)

In [6]:
##filtering out May 2020 records
df_filtered = df_input.filter(F.col('customer_placed_order_datetime').between('2020-05-01 00:00:00','2020-05-31 23:59:59')).selectExpr('customer_placed_order_datetime','restaurant_id','consumer_id','order_total')
#df_filtered.show(10)

In [7]:
##total_order for each restraunt
df_order_res = df_filtered.groupBy('restaurant_id').sum('order_total').orderBy('sum(order_total)').select(col('restaurant_id'),col('sum(order_total)').alias('total_revenue'))
#df_order_res.show(10)

In [8]:
##getting bottom 2% by using ntile function which evenly distributes the cost for repsective buckets
df_order_res.createOrReplaceTempView('fb')
df_output = spark.sql('select restaurant_id,total_revenue from (select restaurant_id,total_revenue, NTILE(50) over(order by total_revenue) as rnk from fb) where rnk <=2')
df_output.show()

+-------------+-------------+
|restaurant_id|total_revenue|
+-------------+-------------+
|           90|        14.16|
|           26|        14.65|
|          298|        15.27|
|           83|        16.33|
+-------------+-------------+

